In [7]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn import preprocessing

In [8]:
df_train = pd.read_csv('train_final.csv')
df_test = pd.read_csv('test_final.csv')
df_test = df_test.drop('ID',axis = 1)

In [9]:
#One Hot Encoding
# for col in ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'sex', 'race', 'native.country']:
#     one_hot_train = pd.get_dummies(df_train[col], prefix=col)
#     one_hot_test = pd.get_dummies(df_test[col], prefix=col)

#     df_train = df_train.drop(col,axis = 1)
#     df_train = pd.concat([df_train,one_hot_train],axis=1)

#     df_test = df_test.drop(col,axis = 1)
#     df_test = pd.concat([df_test,one_hot_test],axis=1)

In [10]:
#Label encoding
label_encoder = preprocessing.LabelEncoder()
for col in ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'sex', 'race', 'native.country']:
    df_test[col] = label_encoder.fit_transform(df_test[col]) 
    df_train[col]= label_encoder.transform(df_train[col]) 

In [11]:
X = df_train.loc[:, df_train.columns != 'income>50K']
y = df_train['income>50K']

X_test_final = df_test.loc[:, df_test.columns != 'income>50K']

In [22]:
estimator = AdaBoostClassifier(
    random_state=42
)

parameters = {
    'n_estimators': [300, 350, 400],
    'learning_rate': [1.5],
}

grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'accuracy',
    cv = 6,
    verbose=True
)

In [23]:
grid_search.fit(X,y)

Fitting 6 folds for each of 3 candidates, totalling 18 fits


GridSearchCV(cv=6, estimator=AdaBoostClassifier(random_state=42),
             param_grid={'learning_rate': [1.5],
                         'n_estimators': [300, 350, 400]},
             scoring='accuracy', verbose=True)

In [24]:
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",grid_search.best_estimator_)
print("\n The best score across ALL searched params:\n",grid_search.best_score_)
print("\n The best parameters across ALL searched params:\n",grid_search.best_params_)

 Results from Grid Search 

 The best estimator across ALL searched params:
 AdaBoostClassifier(learning_rate=1.5, n_estimators=350, random_state=42)

 The best score across ALL searched params:
 0.8726801999863055

 The best parameters across ALL searched params:
 {'learning_rate': 1.5, 'n_estimators': 350}


In [26]:
best_params =  {'learning_rate': 1.5, 'n_estimators': 350}

In [27]:
clf = AdaBoostClassifier(**best_params)
clf.fit(X, y)

AdaBoostClassifier(learning_rate=1.5, n_estimators=350)

In [28]:
y_pred = clf.predict(X_test_final)

In [ ]:
import csv
ctr = 1
data = [['ID', 'Prediction']]
for pred in y_pred:
    row = [ctr, pred]
    data.append(row)
    ctr += 1
# open the file in the write mode
with open('results_adaboost.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(data)